In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel
from langchain_core.tracers.context import collect_runs
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

## Chaining invocations

In [4]:
prompt = PromptTemplate(
    template="Tell me a joke about {topic}"
)

In [5]:
parser = StrOutputParser()

In [6]:
parser.invoke(
    llm.invoke(
        prompt.invoke(
            {"topic": "Python"}
        )
    )
)

'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs! 🐍✨'

## Runnables

Runnables can be 
- executed
    - invoke(), 
    - batch() 
    - and stream()
- inspected,
- and composed

In [7]:
runnables = [prompt, llm, parser]

**Execute methods**

In [8]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINVOKE: {repr(runnable.invoke)}")
    print(f"\tBATCH: {repr(runnable.batch)}")
    print(f"\tSTREAM: {repr(runnable.stream)}\n")

PromptTemplate
	INVOKE: <bound method BasePromptTemplate.invoke of PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')>
	BATCH: <bound method Runnable.batch of PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')>
	STREAM: <bound method Runnable.stream of PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')>

ChatOpenAI
	INVOKE: <bound method BaseChatModel.invoke of ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12025b790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11559cdd0>, root_client=<openai.OpenAI object at 0x117ed64d0>, root_async_client=<openai.AsyncOpenAI object at 0x120471710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))>
	BATCH: <b

**Inspect**

In [9]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINPUT: {repr(runnable.get_input_schema())}")
    print(f"\tOUTPUT: {repr(runnable.get_output_schema())}")
    print(f"\tCONFIG: {repr(runnable.config_schema())}\n")

PromptTemplate
	INPUT: <class 'langchain_core.utils.pydantic.PromptInput'>
	OUTPUT: <class 'langchain_core.prompts.prompt.PromptTemplateOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.PromptTemplateConfig'>

ChatOpenAI
	INPUT: <class 'langchain_openai.chat_models.base.ChatOpenAIInput'>
	OUTPUT: <class 'langchain_openai.chat_models.base.ChatOpenAIOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.ChatOpenAIConfig'>

StrOutputParser
	INPUT: <class 'langchain_core.output_parsers.string.StrOutputParserInput'>
	OUTPUT: <class 'langchain_core.output_parsers.string.StrOutputParserOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.StrOutputParserConfig'>



**Config**

In [10]:
with collect_runs() as run_collection:
    result = llm.invoke(
        "Hello", 
        config={
            'run_name': 'demo_run', 
            'tags': ['demo', 'lcel'], 
            'metadata': {'lesson': 2}
        }
    )

In [11]:
run_collection.traced_runs

[RunTree(id=721e8109-617f-43a5-bde4-2182c0ecb3e1, name='demo_run', run_type='llm', dotted_order='20250314T200800260294Z721e8109-617f-43a5-bde4-2182c0ecb3e1')]

In [12]:
run_collection.traced_runs[0].dict()

{'id': UUID('721e8109-617f-43a5-bde4-2182c0ecb3e1'),
 'name': 'demo_run',
 'start_time': datetime.datetime(2025, 3, 14, 20, 8, 0, 260294, tzinfo=datetime.timezone.utc),
 'run_type': 'llm',
 'end_time': datetime.datetime(2025, 3, 14, 20, 8, 0, 994562, tzinfo=datetime.timezone.utc),
 'extra': {'invocation_params': {'model': 'gpt-4o-mini',
   'model_name': 'gpt-4o-mini',
   'stream': False,
   'temperature': 0.0,
   '_type': 'openai-chat',
   'stop': None},
  'options': {'stop': None},
  'batch_size': 1,
  'metadata': {'lesson': 2,
   'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0}},
 'error': None,
 'serialized': {'lc': 1,
  'type': 'constructor',
  'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
  'kwargs': {'model_name': 'gpt-4o-mini',
   'temperature': 0.0,
   'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}},
  'name': 'ChatOpenAI'},
 'events': [{'name': 'start',
   'time': datetime.d

**Compose Runnables**

In [13]:
chain = RunnableSequence(prompt, llm, parser)

In [14]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [15]:
chain.invoke({"topic": "Python"})

'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [17]:
for chunk in chain.stream({"topic": "Python"}):
    print(chunk, end="", flush=True)

Why do Python programmers prefer dark mode?

Because light attracts bugs!

In [18]:
chain.batch([
    {"topic": "Python"},
    {"topic": "Data"},
    {"topic": "Machine Learning"},
])

['Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs! 🐍✨',
 'Why did the data break up with the database?\n\nBecause it found too many "issues" in their relationship!',
 'Why did the neural network break up with the decision tree?\n\nBecause it found someone with more layers!']

In [19]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


**Turn any function into a runnable**

In [20]:
def double(x:int)->int:
    return 2*x

In [21]:
runnable = RunnableLambda(double)
runnable.invoke(2)

4

**Parallel Runnables**

In [22]:
parallel_chain = RunnableParallel(
    double=RunnableLambda(lambda x: x * 2),
    triple=RunnableLambda(lambda x: x * 3),
)

In [23]:
parallel_chain.invoke(3)

{'double': 6, 'triple': 9}

In [24]:
parallel_chain.get_graph().print_ascii()

+------------------------------+   
| Parallel<double,triple>Input |   
+------------------------------+   
           **        **            
         **            **          
        *                *         
  +--------+          +--------+   
  | Lambda |          | Lambda |   
  +--------+          +--------+   
           **        **            
             **    **              
               *  *                
+-------------------------------+  
| Parallel<double,triple>Output |  
+-------------------------------+  


## LCEL

In [25]:
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')

In [26]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12025b790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11559cdd0>, root_client=<openai.OpenAI object at 0x117ed64d0>, root_async_client=<openai.AsyncOpenAI object at 0x120471710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [27]:
parser

StrOutputParser()

In [28]:
chain = RunnableSequence(prompt, llm, parser)
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12025b790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11559cdd0>, root_client=<openai.OpenAI object at 0x117ed64d0>, root_async_client=<openai.AsyncOpenAI object at 0x120471710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [29]:
prompt | llm | parser

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12025b790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11559cdd0>, root_client=<openai.OpenAI object at 0x117ed64d0>, root_async_client=<openai.AsyncOpenAI object at 0x120471710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [30]:
chain = prompt | llm | parser

In [31]:
chain.invoke(
    {"topic": "computer"}
)

'Why did the computer go to therapy?\n\nBecause it had too many bytes from its past!'